# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 50 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.15699


extracting tarball to tmp_2210.15699...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.15705


extracting tarball to tmp_2210.15705...

...

/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2210.16112/astro_paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'sections/appendix' from 'tmp_2210.16112/sections/appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'sections/conclusion' from 'tmp_2210.16112/sections/conclusion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x

extracting tarball to tmp_2210.16130... done.
Retrieving document from  https://arxiv.org/e-print/2210.16275


extracting tarball to tmp_2210.16275...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2210.16275/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.15699-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.15699) | **Spectroscopy from Lyman alpha to [O III] 5007 of a Triply Imaged  Magnified Galaxy at Redshift z = 9.5**  |
|| Hayley Williams, et al. -- incl., <mark>Yu-Heng Lin</mark> |
|*Appeared on*| *2022-10-31*|
|*Comments*| **|
|**Abstract**| Given their extremely faint apparent brightness, the nature of the first galaxies and how they reionized the Universe's gas are not yet understood. Here we report the discovery, in James Webb Space Telescope (JWST) imaging, of a highly magnified, low-mass (log(M_*/M_sol)=7.70^{+0.11}_{-0.09}) galaxy visible when the Universe was only 510 Myr old, and follow-up prism spectroscopy of the galaxy extending from Lyman alpha to [O III] 5007 in its rest frame. Our JWST spectrum provides [O III] 5007 and H beta detections with a respective signal-to-noise ratio (S/N) of 33 and 7, as well as four additional lines with S/N > 2. These emission lines yield a redshift of z=9.51 and star-formation rate of 0.26^{+0.07}_{-0.05} M_sol per year. The galaxy's large inferred value of [O III]/[O II] = 16 +- 6 suggests that this galaxy has an escape fraction of ionizing radiation larger than 10%, indicating that a population of similar objects could contribute substantially to the reionization budget. Using multiple techniques, we infer a gas oxygen abundance of 12 + log(O/H) = 7.47 +- 0.10 dex, consistent within 2 sigma of the mass-metallicity relation observed for dwarf galaxies in the local Universe. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.15705-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.15705) | **Distorted-Toroidal Flux Rope model for Heliospheric Flux Ropes**  |
|| Teresa Nieves-Chinchilla, Miguel Angel Hidalgo, <mark>Hebe Cremades</mark> |
|*Appeared on*| *2022-10-31*|
|*Comments*| *19 pages, 8 figures*|
|**Abstract**| The three-dimensional characterization of magnetic flux-ropes observed in the heliosphere has been a challenging task for decades. This is mainly due to the limitation to infer the 3D global topology and the physical properties from the 1D time series from any spacecraft. To advance our understanding of magnetic flux-ropes whose configuration departs from the typical stiff geometries, here we present the analytical solution for a 3D flux-rope model with an arbitrary cross-section and a toroidal global shape. This constitutes the next level of complexity following the elliptic-cylindrical (EC) geometry. The mathematical framework was established by Nieves-Chinchilla et al. (2018) ApJ, with the EC flux-rope model that describes the magnetic topology with elliptical cross-section as a first approach to changes in the cross-section. In the distorted-toroidal flux rope model, the cross-section is described by a general function. The model is completely described by a non-orthogonal geometry and the Maxwell equations can be consistently solved to obtain the magnetic field and relevant physical quantities. As a proof of concept, this model is generalized in terms of the radial dependence of current density components. The last part of this paper is dedicated to a specific function, $F(\varphi)=\delta(1-\lambda\cos\varphi)$, to illustrate possibilities of the model. This model paves the way to investigate complex distortions of the magnetic structures in the solar wind. Future investigations will in-depth explore these distortions by analyzing specific events, the implications in the physical quantities, such as magnetic fluxes, heliciy or energy, and evaluating the force balance with the ambient solar wind that allows such distortions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.15768-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.15768) | **Density and infrared band strength of interstellar carbon monoxide (CO)  ice analogues**  |
|| Cristóbal González Díaz, et al. -- incl., <mark>Hector Carrascosa</mark> |
|*Appeared on*| *2022-10-31*|
|*Comments*| *21 pages, 9 figures*|
|**Abstract**| The motivation to study experimentally CO ice under mimicked interstellar conditions is supported by the large CO gas abundances and ubiquitous presence of CO in icy grain mantles. Upon irradiation in its pure ice form, this highly stable species presents a limited ion and photon-induced chemistry, and an efficient non-thermal desorption. Using infrared spectroscopy, single laser interference, and quadrupole mass spectrometry during CO ice deposition, the CO ice density was estimated as a function of deposition temperature. Only minor variations in the density were found. The proposed methodology can be used to obtain the density of other ice components at various deposition temperatures provided that this value of the density is known for one of these temperatures, which is typically the temperature corresponding to the crystalline form. The apparent tendency of the CO ice density to decrease at deposition temperatures below 14 K is in line with recently published colorimetric measurements. This work allowed to revisit the value of the infrared band strength needed for calculation of the CO ice column density in infrared observations, $8.7 \times 10^{-18} ~ {\rm cm ~ molecule}^{-1}$ at 20 K deposition temperature. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.16112-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.16112) | **Red riding on hood: Exploring how galaxy colour depends on environment**  |
|| Pankaj C. Bhambhani, et al. -- incl., <mark>M. A. Lara-Lopez</mark> |
|*Appeared on*| *2022-10-31*|
|*Comments*| *submitted to MNRAS; 15 pages; 12 figures; 1 table; comments welcome*|
|**Abstract**| Galaxy populations are known to exhibit a strong colour bimodality, corresponding to blue star-forming and red quiescent subpopulations. The relative abundance of the two populations has been found to vary with stellar mass and environment. In this paper, we explore the effect of environment considering different types of measurements. We choose a sample of 49, 911 galaxie with $0.05 < z < 0.18$ from the Galaxy And Mass Assembly survey. We study the dependence of the fraction of red galaxies on different measures of the local environment as well as the large-scale "geometric" environment arXiv:1412.2141 [astro-ph.CO] defined by density gradients in the surrounding cosmic web. We find that the red galaxy fraction varies with the environment independently of stellar mass. The local environmental measures exhibit a larger variation in red fraction from lower to higher density, as compared to the geometric environment. By comparing the different environmental densities pairwise, we show that no density measurement fully explains the observed environmental red fraction variation, suggesting the different densities contain different information. We test whether the local environmental measures, when combined together, can explain all the observed environmental red fraction variation. The geometric environment has a small residual effect, and this effect is larger for voids than any other type of geometric environment. This could provide a test of the physics applied to cosmological-scale galaxy evolution simulations as it combines large-scale effects with local environmental impact. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.16130-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.16130) | **Prospects for Constraining the Yukawa Gravity with Pulsars around  Sagittarius A***  |
|| Yiming Dong, et al. -- incl., <mark>Lijing Shao</mark> |
|*Appeared on*| *2022-10-31*|
|*Comments*| *17 pages, 5 figures*|
|**Abstract**| The discovery of radio pulsars (PSRs) around the supermassive black hole (SMBH) in our Galactic Center (GC), Sagittarius A* (Sgr A*), will have significant implications for tests of gravity. In this paper, we predict restrictions on the parameters of the Yukawa gravity by timing a pulsar around Sgr A* with a variety of orbital parameters. Based on a realistic timing accuracy of the times of arrival (TOAs), $\sigma_{\rm TOA}=100\,\mu{\rm s}$, and using a number of 960 TOAs in a 20-yr observation, our numerical simulations show that the PSR-SMBH system will improve current tests of the Yukawa gravity when the range of the Yukawa interaction varies between $10^{1}$-$10^{4}\,{\rm AU}$, and it can limit the graviton mass to be $m_g \lesssim 10^{-24}\,{\rm eV}/c^2$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.16275-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.16275) | **The THESAN project: Lyman-alpha emitter luminosity function calibration**  |
|| Clara Xu, et al. -- incl., <mark>Aaron Smith</mark>, <mark>Lars Hernquist</mark> |
|*Appeared on*| *2022-10-31*|
|*Comments*| *17 pages, 15 figures. Comments welcome. Please visit www.thesan-project.com for more details*|
|**Abstract**| The observability of Lyman-alpha emitting galaxies (LAEs) during the Epoch of Reionization can provide a sensitive probe of the evolving neutral hydrogen gas distribution, thus setting valuable constraints to distinguish different reionization models. In this study, we utilize the new THESAN suite of large-volume (95.5 cMpc) cosmological radiation-hydrodynamic simulations to directly model the Ly$\alpha$ emission from individual galaxies and the subsequent transmission through the intergalactic medium. THESAN combines the AREPO-RT radiation-hydrodynamic solver with the IllustrisTNG galaxy formation model and includes high- and medium-resolution simulations designed to investigate the impacts of halo-mass-dependent escape fractions, alternative dark matter models, and numerical convergence. We find important differences in the Ly$\alpha$ transmission based on reionization history, bubble morphology, frequency offset from line centre, and galaxy brightness. For a given global neutral fraction, Ly$\alpha$ transmission reduces slightly when low mass haloes dominate reionization over high mass haloes. Furthermore, the variation across sightlines for a single galaxy is greater than the variation across all galaxies. This collectively affects the visibility of LAEs, directly impacting observed Ly$\alpha$ luminosity functions (LFs). We employ Gaussian Process Regression using SWIFTEmulator to rapidly constrain an empirical model for dust escape fractions and emergent spectral line profiles to match observed LFs. We find that dust strongly impacts the Ly$\alpha$ transmission and covering fractions of $M_{UV} < -19$ galaxies in $M_{vir} > 10^{11} {\rm M}_{\odot}$ haloes, such that the dominant mode of removing Ly$\alpha$ photons in non-LAEs changes from low IGM transmission to high dust absorption around $z \sim 7$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.16014-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.16014) | **Kinetic field theory: Generic effects of alternative gravity theories on  non-linear cosmic density-fluctuations**  |
|| Alexander Oestreicher, et al. -- incl., <mark>Lavinia Heisenberg</mark> |
|*Appeared on*| *2022-10-31*|
|*Comments*| *Submitted to JCAP*|
|**Abstract**| Non-linear cosmic structures contain valuable information on the expansion history of the background space-time, the nature of dark matter, and the gravitational interaction. The recently developed kinetic field theory of cosmic structure formation (KFT) allows to accurately calculate the non-linear power spectrum of cosmic density fluctuations up to wave numbers of $k\lesssim10\,h\,\mathrm{Mpc}^{-1}$ at redshift zero. Cosmology and gravity enter this calculation via two functions, viz. the background expansion function and possibly a time-dependent modification of the gravitational coupling strength. The success of the cosmological standard model based on general relativity suggests that cosmological models in generalized theories of gravity should have observable effects differing only weakly from those in standard cosmology. Based on this assumption, we derive the functional, first-order Taylor expansion of the non-linear power spectrum of cosmic density fluctuations obtained from the mean-field approximation in KFT in terms of the expansion function and the gravitational coupling strength. This allows us to study non-linear power spectra expected in large classes of generalized gravity theories. To give one example, we apply our formalism to generalized Proca theories. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.15983-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.15983) | **Re-investigating stellar, solar and galactic spectral modulations:  rapidly oscillating spacetime effects due to axions or numerical ghosts?**  |
|| Fabrizio Tamburini, <mark>Ignazio Licata</mark> |
|*Appeared on*| *2022-10-31*|
|*Comments*| *9 pages, 1 table*|
|**Abstract**| In our previous work [Tamburini and Licata (2017)] we discussed the hypothesis that the ultrafast periodic spectral modulations with frequency $f_S \simeq 0.61$ THz found by Borra and Trottier (2016) in $236$ main sequence stars from a sample of $2.5$ million spectra of galactic halo stars of the Sloan Digital Sky Survey were due to axion-like dark matter piled up in the center of these stars. These temporary matter/dark matter structures are characterized by a spacetime geometry rapidly oscillating at frequencies that depend on the axion mass $m_a$ [Brito {\it et~al.} (2015); Brito {\it et~al.} (2016)]. Borra (2013) found two additional frequencies, $ f_{1,G} \simeq 9.5$ THz and $f_{2,G} \simeq 8.9$ THz, in the Sloan dataset of galaxies, redshifted by the cosmological expansion and, for any redshift value, $f_S + f_{2,G} = f_{1,G}$ is found. Hippke (2019) showed that $f_{2,G}$ is spurious and introduced by the data analysis procedure due to the nonrandom spacing of the spectral absorption lines. This was not even found by Isaacson (2019) when re-observed four of these stars with different instrumentation and data reduction procedure. Interestingly, Hippke found $f_S$ in the solar spectrum but not in the Kurucz (2005) artificial solar spectrum whilst its spectral power estimated by Isaacson resulted below the accepted error $(1\%)$. From these results, we discuss the validity of our ansatz by analyzing the common features present in all the spectra. In the worst case, if all the three frequencies are not real the oscillating axion core models is not valid. Assuming, instead, the validity of $f_S$ from the results from the analysis of the solar spectra, those oscillating modes may be transient modes favoring the axion hypothesis in the mass range $(10^{- 3} < m_a < 2.4 \times 10^{3})~ \mathrm{\mu eV}$, also according to the recent limits from the gamma ray burst GRB221009A. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error invalid header</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

73  publications in the last 7 days.
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers